In [16]:
from ultralytics import YOLO
import cv2
import os

In [17]:
VEHICLE_CLASSES = {2, 3, 5, 7}

vehicle_model = YOLO('../yolov8n.pt')
plate_model = YOLO('../detection-plate/runs/detect/train/weights/best.pt')

In [18]:
def detect_plates_on_vehicles(image_path, out_path=None):
    img = cv2.imread(image_path)
    # Stage 1: detect vehicles
    v_results = vehicle_model(img)[0]
    for box in v_results.boxes:
        cls = int(box.cls)
        if cls not in VEHICLE_CLASSES:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        # Optional: add margin to the crop
        pad = 10
        x1m, y1m = max(0, x1 - pad), max(0, y1 - pad)
        x2m, y2m = min(img.shape[1], x2 + pad), min(img.shape[0], y2 + pad)
        crop = img[y1m:y2m, x1m:x2m]

        # Stage 2: detect plates in the cropped vehicle
        p_results = plate_model(crop)[0]
        for pbox in p_results.boxes:
            px1, py1, px2, py2 = map(int, pbox.xyxy[0])
            # Map plate coords back to original image
            ox1, oy1 = px1 + x1m, py1 + y1m
            ox2, oy2 = px2 + x1m, py2 + y1m
            # Draw plate bbox
            cv2.rectangle(img, (ox1, oy1), (ox2, oy2), (0, 255, 0), 2)
            cv2.putText(
                img,
                f"plate {pbox.conf[0]:.2f}",
                (ox1, oy1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1,
            )

    # Draw vehicle boxes (optional)
    for box in v_results.boxes:
        cls = int(box.cls)
        if cls in VEHICLE_CLASSES:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(
                img,
                f"{vehicle_model.names[cls]} {box.conf[0]:.2f}",
                (x1, y2 + 15),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),
                1,
            )

    if out_path:
        cv2.imwrite(out_path, img)
    return img

In [19]:
TEST_DIR="../test/images"
TEST_RESULTS_DIR="./test-results"

os.makedirs(TEST_RESULTS_DIR, exist_ok=True)

for fname in os.listdir(TEST_DIR):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue
    in_path = os.path.join(TEST_DIR, fname)
    out_path = os.path.join(TEST_RESULTS_DIR, fname)

    result = detect_plates_on_vehicles(in_path, out_path)

    out_file = os.path.join(TEST_RESULTS_DIR, os.path.basename(fname))
    cv2.imwrite(out_file, result)
    print(f"[OK] Wrote {out_path}")


0: 384x640 4 persons, 1 car, 1 truck, 36.5ms
Speed: 1.4ms preprocess, 36.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 416x640 1 License_Plate, 34.4ms
Speed: 0.9ms preprocess, 34.4ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 640)

0: 544x640 (no detections), 46.9ms
Speed: 1.1ms preprocess, 46.9ms inference, 0.2ms postprocess per image at shape (1, 3, 544, 640)
[OK] Wrote ./test-results/Japanese-20License-20Plates-20-20Custom-20Japanese-20License-20P-20-1-_jpg.rf.511b035c29367e861cc81a4df187bc41.jpg

0: 384x640 1 car, 32.3ms
Speed: 0.8ms preprocess, 32.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 33.2ms
Speed: 0.8ms preprocess, 33.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
[OK] Wrote ./test-results/Cars430_png_jpg.rf.ed89a6a4fb4468882713f4b134a69e8e.jpg

0: 448x640 5 persons, 3 cars, 1 bus, 37.0ms
Speed: 0.6ms preprocess, 37.0ms inference, 0.6ms postprocess per im

In [21]:
VIDEO_IN = "./video.mp4"
VIDEO_OUT = "./video_out.mp4"

os.makedirs(os.path.dirname(VIDEO_OUT), exist_ok=True)